<a href="https://colab.research.google.com/github/mohammadreza-mohammadi94/NLP-Projects/blob/main/NLP-TextMorph/NLP_Text_Morph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries & Project Setup

In [1]:
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from typing import List, Tuple
import pandas as pd
import os

import logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)",
)
logger = logging.getLogger(__name__)

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load NLTK Packages
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

# Text Processor

In [3]:
class TextProcessor:
    """ Used to process texts using NLTK """
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words("english"))

    def  tokenize_text(self, text: str):
        """ For tokenization sentences """
        try:
            tokens = word_tokenize(text.lower())
            logger.info("Tokenization Succeed..")
            return tokens
        except Exception as e:
            logger.error(f"Tokenization Failed: {e}")

    def pos_tag_tokens(self, tokens: List[str]):
        """Tags the tokens"""
        try:
            tagged = pos_tag(tokens)
            logger.info("Tagging Succeed...")
            return tagged
        except Exception as e:
            logger.error(f"Tagging Failed: {e}")
            return []

    @staticmethod
    def get_wordnet_pos(treebank_tag: str):
        """ Converts Treebank tags to Wordnet """
        pos_mapping = {
            'V': 'v',  # فعل
            'N': 'n',  # اسم
            'J': 'a',  # صفت
            'R': 'r'   # قید
        }
        return pos_mapping.get(treebank_tag[0], 'n')

    def lemmatize_with_pos(self, tagged_tokens: List[Tuple[str, str]]) -> List[str]:
        """Lemmatization Using POS"""
        try:
            lemmatized = [self.lemmatizer.lemmatize(word, pos=self.get_wordnet_pos(tag))
                         for word, tag in tagged_tokens]
            logger.info("Lemmatized Using POS Succeed....")
            return lemmatized
        except Exception as e:
            logger.error(f"Lemmatization With POS Failed: {e}")
            return []

    def lemmatize_without_pos(self, tokens) -> List[str]:
        """Lemmatization without POS (Default value 'n' for pos)"""
        try:
            lemmatized = [self.lemmatizer.lemmatize(word) for word in tokens]
            logger.info("Lemmatization Without POS Succeed...")
            return lemmatized
        except Exception as e:
            logger.error(f"Lemmatization Without POS Failed: {e}")
            return []

    def remove_stopwords(self, tokens):
        return [token for token in tokens if token not in self.stop_words and token.isalpha()]

    def process_text(self, text: str):
        """Processing text and shows results"""
        # Tokenization
        tokens = self.tokenize_text(text)
        if not tokens:
            return {}

        # Remove Stopwords
        filtered_tokens = self.remove_stopwords(tokens)

        # POS Tagging
        tagged_tokens = self.pos_tag_tokens(filtered_tokens)

        # Lemmatization
        lemmatized_with_pos = self.lemmatize_with_pos(tagged_tokens)
        lemmatized_without_pos = self.lemmatize_without_pos(filtered_tokens)

        # Results
        results = {
            'Original Tokens': filtered_tokens,
            'POS Tags': [tag for _, tag in tagged_tokens],
            'Lemmatized with POS': lemmatized_with_pos,
            'Lemmatized without POS': lemmatized_without_pos
        }
        return results

# Use TextProcessor

In [4]:
def display_results(results: dict, filepath: str = "results.csv", overwrite: bool = True) -> str:
    """
    Displays results in a DataFrame and saves it to a file.

    Args:
        results (dict): Dictionary containing text processing results.
        filepath (str): Path to save the DataFrame (default: 'results.csv').
        overwrite (bool): Whether to overwrite the file if it exists (default: True).

    Returns:
        str: Formatted string representation of the DataFrame.
    """
    if not results:
        logger.warning("Nothing to show or save...")
        return ""

    try:
        # Creating DataFrame from results
        df = pd.DataFrame(results)
        logger.info("DataFrame created successfully")

        # Save Df
        if os.path.exists(filepath) and not overwrite:
            logger.warning(f"File '{filepath}' already exists and overwrite is False. Skipping save.")
        else:
            df.to_csv(filepath, index=False, encoding='utf-8')
            logger.info(f"Results saved to '{filepath}' successfully")

        # Show df
        result_string = df.to_string(index=False)
        print(result_string)

        return result_string

    except Exception as e:
        logger.error(f"Failed to process or save results: {e}")
        return ""

In [6]:
def main():
    sample_text = """
            Reasoning models like OpenAI’s o1 have shown great promise in domains like math and physics,
            thanks to their ability to effectively fact-check themselves by working through complex problems step by step.
            This reasoning comes at a cost, however: higher computing and latency.
            That’s why labs like Anthropic are pursuing “hybrid” model architectures that combine reasoning components with standard, non-reasoning elements.
            Hybrid models can quickly answer simple questions while spending additional time considering more challenging queries.
                """
    logger.info("Text Processing Started...")

    # Create instance of TextProcessor
    processor = TextProcessor()

    # Processing
    results = processor.process_text(sample_text)

    # Show results
    print(display_results(results))

if __name__ == '__main__':
    main()

Original Tokens POS Tags Lemmatized with POS Lemmatized without POS
      reasoning      VBG              reason              reasoning
         models      NNS               model                  model
           like       IN                like                   like
         openai       JJ              openai                 openai
          shown      VBN                show                  shown
          great       JJ               great                  great
        promise       NN             promise                promise
        domains      NNS              domain                 domain
           like       IN                like                   like
           math       NN                math                   math
        physics      NNS              physic                 physic
         thanks      NNS              thanks                 thanks
        ability       NN             ability                ability
    effectively       RB         effectively    